## GENERATE THE DATA

In [ ]:
import numpy, os, sys, subprocess,re, time, socket

In [ ]:
from pathlib import Path
import configparser
config = configparser.ConfigParser()
config.read("../config.ini")
prism_path = config.get("paths", "prism_path")
data_path = config.get("paths", "data")

due to large time consumed made separately for each pop_size

In [ ]:
folders = ["models"]
model_types = ["synchronous_parallel_"]
n_samples = [3500,1500,100]
populations = [2,3,5,10]
dimension_sample_size = 5

generate param values:

In [ ]:
#p_values=[]
#for v_p in numpy.random.uniform(0.0, 1.0,dimension_sample_size):
#    p_values.append(v_p)
#q_values=[]
#for v_q in numpy.random.uniform(0.0, 1.0,dimension_sample_size):
#    q_values.append(v_q)

In [ ]:
p_values = [0.028502714675268215, 0.45223461506339047, 0.8732745414252937, 0.6855555397734584, 0.13075717833714784]

In [ ]:
q_values = [0.5057623641293089  , 0.29577906622244676, 0.8440550299528644, 0.8108008054929994, 0.03259111103419188]

In [ ]:
cwd = os.getcwd()
os.chdir("..")
sys.path.append(os.getcwd())
os.chdir(cwd)
from src.mc_prism import call_prism

In [ ]:
cwd=os.getcwd()
cwd

Here we generate the data by simulating the model:

In [ ]:
# max_sample = max(n_samples)

# start_time = time.time()
# i=1
# Experiments={}
# for folder in folders:
#     for model_type in model_types:
#         if model_type == "synchronous_parallel_":
#             sim_lenght=2
#         for N in populations:
#             if model_type == "semisynchronous_parallel_":
#                 sim_lenght= 2*N
#             if model_type == "asynchronous_parallel_":
#                 sim_lenght= 2*N
#             model = model_type+str(N)+".pm"
#             for v_p in p_values:
#                 for v_q in q_values:
#                     Experiments[(model_type,N,max_sample,v_p,v_q)]=[]
#                     #file = open("path_{}_{}_{}_{}_{}.txt".format(model_type,N,n_sample,v_p,v_q),"w+")
#                     #file.close()
#                     for sample in range(max_sample):
#                         if i==1:
#                             ## dummy path file for prism output
#                             path_file=f"path_{model_type}{N}_{max_sample}_{v_p}_{v_q}.txt"
#                             ## here is the PRISM called
#                             call_prism(f"{model} -const p={v_p},q={v_q} -simpath {str(sim_lenght)} {path_file}", silent=True, prism_output_path=cwd)
#                             ## parse the dummy file
#                             file = open(path_file,"rt")
#                             last_line = file.readlines()[-1]
#                             #print(last_line)
#                             ## close and remove dummy file
#                             file.close()  
#                             os.remove(path_file)
#                             ## append the experiment
#                             Experiments[(model_type,N,max_sample,v_p,v_q)].append(sum( list(map(lambda x: int(x), last_line.split(" ")[2:-1]))))
#                         #i=0
#                     #print(model_type,N,max_sample,v_p,v_q)
#                     print(Experiments[(model_type,N,max_sample,v_p,v_q)])
#                     for n_sample in n_samples:
#                         if n_sample is not max_sample:
#                             Experiments[(model_type,N,n_sample,v_p,v_q)] = Experiments[(model_type,N,max_sample,v_p,v_q)][:n_sample]
#                             print(Experiments[(model_type,N,n_sample,v_p,v_q)])
# print("  It took", socket.gethostname(), time.time() - start_time, "seconds to run")

This took Freya 6 days to run

As you can see this process took a while so we have pickled the data and we load it back

In [ ]:
#Experiments

In [ ]:
import pickle

In [ ]:
#pickle.dump( D,open(os.path.join(data_path,"Experiments_two_param.p", "wb" )))

In [ ]:
Experiments = pickle.load(open(os.path.join(data_path,"Experiments_two_param.p"), "rb" ))

In [ ]:
Experiments[('synchronous_parallel_',10,100,0.028502714675268215,0.5057623641293089)]

Here we compute frequency - hence estimated probability of reaching respective bSCC

In [ ]:
Data={}
for folder in folders:
    for model_type in model_types:
        for N in populations:
            model = os.path.join(folder,model_type+str(N)+".pm")
            for v_p in p_values:
                for v_q in q_values:
                    for n_sample in n_samples:
                        Data[(model_type,N,n_sample,v_p,v_q)]=[]
                        for i in range(N+1):
                            Data[(model_type,N,n_sample,v_p,v_q)].append(len(list(filter(lambda x: x ==i ,Experiments[(model_type,N,n_sample,v_p,v_q)])))/n_sample)

In [ ]:
Data[('synchronous_parallel_',10,100,0.028502714675268215,0.5057623641293089)]

In [ ]:
pickle.dump(Data,open(os.path.join(data_path,"Experiments_freq_two_param.p"), "wb"))

This notebook has been checked by: 

`add name here` on `add date here`